CAPSTONE PROJECT III: Recurrent Neural Networks for sentiment analysis

In [43]:
# Import dependencies
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Embedding

This project uses the Large Movie Review Dataset of 50,000 reviews from IMDb. It contains an even number of positive
and negative reviews. Keras has this data set built-in, which we import below.

In [44]:
# Get the dataset
from keras.datasets import imdb

In [45]:
# Set the number of words and the maximum length of a review
num_words = 5000
maxlen = 500

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

In [ ]:
# Load the data
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [ ]:
# View a review and it's label
print('+++++++++Review+++++++++')
print(X_train[6])
print('+++++++++Labels+++++++++')
print(y_train[6])

The Data is preprocessed - all the words have been mapped to integers according to their frequency. A value of 0 is used for padding. The label is either 0 - negative or 1 - positive.

All reviews need to be the same length, thus reviews shorter than 500 words need to be padded with zeros.

In [46]:
# Process the data to ensure all inputs are 500 values using pad_sequences
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

BUILDING THE MODEL: 
The Embedding layer is the first hidden layer of a network. The first argument is input_dim: This is the size of the vocabulary in the text data. It is thus 5000. The second is output_dim: This defines the size of the output vectors from this layer for each word. We will use 32, as it was found to be the best value for this problem.The last argument is input_length: This is the length of input sequences, which all consist of 500 words.

In [49]:
# Build the model
model = Sequential()

# The embedding layer (input dim=5000, output dim=32 and input length=500)
model.add(Embedding(input_dim=num_words, output_dim=32, input_length=maxlen))
model.add(Dropout(0.4))

# The LSTM model
model.add(LSTM(64))

# The dense layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 32)           320000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 344,897
Trainable params: 344,897
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Fit the model using 4 epochs, a batch size of 100 and a verbose value of 2
model.fit(X_train, y_train, epochs=4, batch_size=100, verbose=2)

Epoch 1/3
250/250 - 101s - loss: 0.4983 - accuracy: 0.7465
Epoch 2/3
250/250 - 101s - loss: 0.2586 - accuracy: 0.9000
Epoch 3/3
250/250 - 101s - loss: 0.2075 - accuracy: 0.9219


In [51]:
# Test the model
scores = model.evaluate(X_test, y_test, verbose=2)
print("Accuracy: ", (scores[1]*100),"%")

782/782 - 38s - loss: 0.3542 - accuracy: 0.8461
Accuracy:  84.60800051689148 %


PREDICT SOMETHING: 
Here we will define a function to take in text, clean it and convert it to integers to be interpreted by the model.
We will then use the outcome to say wether or not the review was positive.

In [52]:
import re

def clean(txt):
# Remove punctuation, return a list of the words
  list_words = []
  txt = txt.split()

  for x in txt:
    z = re.findall("[a-zA-Z]", x)
    list_words.append(''.join(z).lower())

  return list_words

clean('This is a sentence!')


['this', 'is', 'a', 'sentence']

In [99]:
def predict_sentiment(sentence, model):
  # Generate a sentiment value for a review  
  word_indexes = []
  list_words = clean(sentence)
  print(list_words)

  for word in list_words:
    # Convert words to integers
    word_indexes.append(imdb.get_word_index()[word])
  
  # Ensure all inputs are 500 values using pad_sequences
  padded_indexes =  sequence.pad_sequences([word_indexes], maxlen=maxlen)

  # Output the outcome
  outcome = model.predict(padded_indexes)
  return outcome

In [104]:
# Hypothetical review and outcome
sentiment = predict_sentiment("The film was unoriginal and boring. I would rather watch something else.", model)

if sentiment >= 0.5:
  print("The review was positive")
else:
  print("The review was negative")


['the', 'director', 'was', 'a', 'fool']
The review was negative
